In [1]:
DATA_NAME = 'standardTesting-full' 
TRANSFORM = 'wavelet-diagonal'
CHANNEL = 'gray'

In [2]:
path_list = DATA_NAME.split("-") + TRANSFORM.split("-")
if CHANNEL:
    path_list.append(CHANNEL)
print(f"Name the notebook:\nindependence_{'_'.join(path_list[::-1])}.ipynb")
FULL_DATA_NAME='-'.join(path_list)


import git
from pathlib import Path
import os
CWD = os.getcwd()
ROOT_DIR = Path(git.Repo('.', search_parent_directories=True).working_tree_dir)

Path(os.path.join(CWD, "CSVs")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "plots")).mkdir(exist_ok=True)
Path(os.path.join(CWD, "cache")).mkdir(exist_ok=True)

GROUP = 'layer' if TRANSFORM.split("-")[0] == 'wavelet' else ('band' if TRANSFORM.split("-")[0] == 'fourier' else 'error')
RERUN = False
SKIP_OPTIMIZE_STEP = False

Name the notebook:
independence_gray_diagonal_wavelet_full_standardTesting.ipynb


In [3]:
os.chdir(os.path.join(ROOT_DIR, "utilities"))
from testing import * # If MATLAB is not installed, open utilities and set to False
from plotting import *
os.chdir(CWD)
np.random.seed(0)

In [4]:
from scipy import spatial
from sklearn.decomposition import PCA

In [5]:
group_data_map = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}.pickle'))
group_total_samples = pd.read_pickle(os.path.join(ROOT_DIR, "transformed-data", f'{FULL_DATA_NAME}-size.pickle'))

In [6]:
if 'fourier' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)#[::3]
elif 'wavelet' in TRANSFORM:
    GROUPS = np.arange(2, sorted(group_data_map)[-1] + 1)
elif 'learned' in TRANSFORM:
    nonskewed_df = pd.read_csv(os.path.join(ROOT_DIR, 'learned-filters', 'nonskewed_filter_idxs_df.csv')).set_index(['dataset', 'num_images', 'num_bootstrap'])
    nonskewed_filter_idxs = eval(nonskewed_df.loc[DATA_NAME].sort_values('num_images', ascending=False)['nonskewed_filter_idxs'].iloc[0]) 
    GROUPS = nonskewed_filter_idxs # can set to filter_group_map.keys() to include all prepared filters

In [7]:
n_bootstrap = int(1e5) 
bootstrap_size = int(1e4)


In [8]:

cov_matrix = np.zeros((len(GROUPS), len(GROUPS)))
for _ in tqdm(range(n_bootstrap)):
    X = np.zeros((bootstrap_size, len(GROUPS)))
    for i in range(len(GROUPS)):
        group = GROUPS[i]
        if len(group_data_map[group]) < 100:
            X[:, i] = np.full(bootstrap_size, np.nan)
        else:
            X[:, i] = np.random.choice(group_data_map[group], size=(bootstrap_size), replace=True)
    cov_matrix += np.cov(X, rowvar=False)
cov_matrix /= n_bootstrap

  0%|          | 0/100000 [00:00<?, ?it/s]

In [9]:



# Convert covariance matrix to DataFrame for better formatting
cov_df = pd.DataFrame(cov_matrix, index=GROUPS, columns=GROUPS)
cov_df = cov_df.dropna(how='all').dropna(axis=1, how='all')
cov_df.to_csv(os.path.join(CWD, "CSVs", f'covariance_matrix.csv'))
cov_df.round(2)

,4,5,6,7,8,9,10
4,357.12,0.00,0.00,0.00,-0.00,-0.00,0.00
5,0.00,82.62,0.00,0.00,-0.00,-0.00,-0.00
6,0.00,0.00,14.35,0.00,-0.00,0.00,-0.00
7,0.00,0.00,0.00,2.54,0.00,-0.00,-0.00
8,-0.00,-0.00,-0.00,0.00,0.54,-0.00,-0.00
9,-0.00,-0.00,0.00,-0.00,-0.00,0.09,0.00
10,0.00,-0.00,-0.00,-0.00,-0.00,0.00,0.01


In [10]:
corr_matrix = cov_df / np.sqrt(np.outer(np.diag(cov_df), np.diag(cov_df)))
corr_matrix.round(5)

,4,5,6,7,8,9,10
4,1.00000,0.00002,0.00003,0.00002,-0.00001,-0.00003,0.00000
5,0.00002,1.00000,0.00003,0.00003,-0.00002,-0.00005,-0.00004
6,0.00003,0.00003,1.00000,0.00003,-0.00001,0.00002,-0.00001
7,0.00002,0.00003,0.00003,1.00000,0.00002,-0.00001,-0.00006
8,-0.00001,-0.00002,-0.00001,0.00002,1.00000,-0.00003,-0.00003
9,-0.00003,-0.00005,0.00002,-0.00001,-0.00003,1.00000,0.00002
10,0.00000,-0.00004,-0.00001,-0.00006,-0.00003,0.00002,1.00000


In [11]:
np.linalg.norm(cov_df - np.diag(np.diag(cov_df)))  # Remove diagonal elements for visualization

0.006854311911495466

In [12]:
pca = PCA()
pca.fit(cov_df)

print("Singular values (explained variance):")
print(pca.explained_variance_)

print("\nPrincipal components (eigenvectors):")
eigenvectors = pca.components_
eigenvectors_df = pd.DataFrame(eigenvectors)
eigenvectors_df

Singular values (explained variance):
[1.82486057e+04 9.47894845e+02 2.74702853e+01 8.08583745e-01
 3.19806024e-02 6.89322290e-04 4.25187532e-36]

Principal components (eigenvectors):


,0,1,2,3,4,5,6
0,0.999152,-0.040612,-0.006622,-0.001170,-0.000247,-0.000042,-0.000005
1,0.040340,0.998517,-0.036024,-0.006148,-0.001299,-0.000221,-0.000027
2,0.007999,0.035397,0.998249,-0.045759,-0.009301,-0.001569,-0.000189
3,0.001752,0.007577,0.044679,0.996193,-0.073502,-0.011718,-0.001411
4,0.000495,0.002139,0.012325,0.071693,0.993480,-0.087181,-0.010053
5,0.000125,0.000540,0.003100,0.017535,0.084255,0.988934,-0.120805
6,0.000030,0.000132,0.000755,0.004267,0.020209,0.119456,0.992624


In [13]:
cos_dist = np.diag(spatial.distance.cdist(eigenvectors, np.eye(cov_df.shape[0]), metric='cosine'))
cos_dist = cos_dist = [np.min(row) for row in cos_dist]
cos_dist


[0.0008476580002390044,
 0.0014833928111550598,
 0.0017514647985434895,
 0.0038065235189047586,
 0.006520341847259137,
 0.011066283442753289,
 0.007375650699567804]